# About this Page
This section is a developer aid; TODO remove later!
*This notebook is intended to align with [storyboard Version 51](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=51)* (more or less)
If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the full Times Square URL you used to get the error. 

## What is new?
- Added "What is new"
- Sunset bug: DM-47511
- Retrieve of records from each source will repeatedly get pages until all data for the specified date range has been retrieved.
- Data Log partitions exposure counts by "quality" flags.
- Data Log now has link to drill down to Detail exposure list.
- This page tries to work around existing problems with APIs used for source data. (There is a ticket to fix the APIs)
- Errors with APIs are display in this page in Red.

## Known issues
- (none known)

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-11-05"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

verbose = "true"  # TODO change to false before push, else true

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

verbose = (verbose == "true") 


In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,                
    limit=5000,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

DBG get_records endpoint='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports'
DBG get_records qstr: is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241106
DEBUG protected_get(url='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241106',timeout=(5.05, 20.0))
DBG get_records-2 len(page)=1 len(recs)=1
DBG get_records-2 len(self.records)=1 status={'endpoint_url': 'https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241106', 'number_of_records': 1, 'error': None}
DEBUG protected_get(url='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments',timeout=(5.05, 20.0))
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTCo

----------

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-05 to 2024-11-06 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-DRAFT)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: Simonyi

```
During the day time we had TMA parked at horizon, mirror cover opened, for mirror cleaning. We turned on the hexapods and rotator, (disabled when not in use) and ran hexapod warmups. Since bump test and HP breakaway test are recommended to be ran in higher elevation and TMA handover was planned within less than time needed for these tests, we decided to do TMA handover first, then move in elevation to 70, do the M1M3 tests, then raise M1M3 before moving TMA further. 

After TMA handover, we moved in az first for day crew to check the oil. Then we moved elevation to 70, and ran HP breakaway test using GUI so we can run multiple HPs simultaneously. All passed, then while switching to CSC mode, M1M3 faulted. While recovering, it faulted again due to clearing interlock happened when M1M3 was not in standby state. Recovered, and after Enabling MTM1M3, we ran bump test. Primary FA 233, 235, 330 failed bump test, but when Petr re-ran individually 233 and 330 passed, and FA 235 also was near the limit and we agreed it will be overshooting. We raised M1M3.

Started TMA Checkout BLOCK-T250, track_target timed out since we put the position too far from original position. It worked when we reran. We sent TMA to el=65, az=160 for Guillem's align test (BLOCK-T88). After quick alignment, we turned laser tracker off by script and put laser tracker in standby.

With Sandrine's help on temperature control, we turned off HVAC around 8:10PM and opened the dome (mirror cover deployed before) when the temperature difference between outside and M2 air temperature was ~2 degrees. 

We started twilight flat (BLOCK-T215), but realized we had mirror cover still covered. We stopped the script, stopped tracking, mirror cover opened, then re-ran the twilight flat. While taking flat images, it went out of focus and we found hexapods jumped without reason (OBS-648).

Then we started BLOCK-T60 manual alignment with offset measured from BLOCK-T88, and compared the spherical zernike and concluded image #84 was the best and best offset values are recorded in the test case. T88-E2123.

Handed over to late night shift.

AOS Summary
The night was very productive thanks to summit team getting the telescope ready early! We continued refining the telescope alignment and understand our measurement noise. We conducted a reduced sensitivity matrix on the hexapods. While doing the hexapod reduced sensitivity matrix, we found a good position that led to our best image quality so far: 1.29 arcsec. We still see trefoil in the in-focus image (0.2 microns). We will use the measured responses to refine the alignment. 

The night was very productive despite LOVE blocking us for 1.5 hours in the middle of the night!  The cause was filling the postgres PV ("disk"), which may have been caused by a mis-configured database (writing voluminous write-ahead logs).

DM Summary
At the end of the night we took a set of c. 24 dithered r-band exposures using the current best AOS settings (sharp images with prominent trefoil). While getting this script working we took a series of non-dithered 15s exposures (206..215,225..223) with comparable image quality which might be of value for snap eradication.  216..219 have a waffle, and are followed by pinhole exposures

Closed Dome test and closing summary
We closed the MTDome around 5am per the current regulation. There was no issue with closing the MTDome shutter tonight (only 1.5min differences between two shutter differences. While ComCam daytime calibrations were taken (BLOCK-T220), M2 closed-loop break-out brake test during TMA slew (BLOCK-241) was proceeded and completed. After calibration completed, we run Az brake test and El brake test (BLOCK-T231 and BLOCK-T240) respectively. GIS triggered with gate at P6 and MTMount and MTRotator only faulted as expected. Starting closing the night around 06:30amCLT/09:30amUTC. Note that el:0deg, M1M3 was fault while slewing in Az direction. Force controller X moment limit. M2hexapod was failed at the same time while moving the origin. It was okay after re-running the script. After then TMA was moving in Az only with static support (disabled and lowered) M1M3.
```

Telescope Status: TMA parked at Az: 148, el: 0.0 with retracted mirror cover as requested. Locking pin engaged, OSS on, PS OFF. Dome is parked at 328 deg. All CSCs were set as requested: MTM1M3 DISABLED - lowered and disabled. MTM2 ENABLED in closed loop, MTRotator and MTHexapods are all origin and 0deg then in STANDBY w/ cabinets and PXIs powered off, MTAOS STANDBY, MTDome DISABLED, MTDomeTrajectory STANDBY, MTMount DISABLED.


*Authors: HyeYun Park, Kshitija Kelkar, Yijung Kang, Chris Walter, Sandrine Thomas, Craig Lage, Robert Lupton, Brian Stalder*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-05**

,(times in UTC),
Moon Illumination,20%,(% illuminated)
Sun Set,2024-11-05 23:05:14,
Evening Civil Twilight,2024-11-05 23:34:56,(-6 degrees)
Evening Nautical Twilight,2024-11-06 00:05:31,(-12 degrees)
Evening Astronomical Twilight,2024-11-06 00:37:24,(-18 degrees)
Moon Set,2024-11-06 03:16:57,
Solar Midnight,2024-11-06 04:26:27,
Morning Astronomical Twilight,2024-11-06 08:15:30,(-18 degrees)
Morning Nautical Twilight,2024-11-06 08:47:24,(-12 degrees)
Morning Civil Twilight,2024-11-06 09:17:59,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## DDV <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

In [10]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.


In [11]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCam,LSSTCam
Total Night,7:38:06,7:38:06
Total Exposure,2:23:48,2:56:50
Slew time(1),0:00:00,0:00:00
Readout time(2),0:11:43,0:29:30
Time loss to fault,NA,NA
Time loss to weather,NA,NA
Idle time,5:14:17,4:41:15
Number of exposures,306,770
Mean readout time,0.000639,0.000639
Number of slews(1),0,0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
- Observing Operations - OBS
- TODO: better tickets using Jira API

In [12]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [obs-647](https://rubinobs.atlassian.net/browse/obs-647)

- [/rubinobs-lfa-cp/LOVE/OLE/2024/11/04/LOVE_OLE_2024-11-05T04:06:17.057.png](https://s3.cp.lsst.org/rubinobs-lfa-cp/LOVE/OLE/2024/11/04/LOVE_OLE_2024-11-05T04:06:17.057.png)

- [obs-611](https://rubinobs.atlassian.net/browse/obs-611)

- [OBS-646](https://rubinobs.atlassian.net/browse/OBS-646)

In [13]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241106) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:green; color:white; font-size:20px">BETA</font>

In [14]:
# Exposure Report
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        if df.empty:
            continue
        print(field_name.title().replace('_',' '))
        display(HTML(df.style.hide().to_html(escape=False)))

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 306 total exposures**

Observation Type


Detail,good,questionable,junk,unknown
acq,5,0,0,75
bias,0,0,0,32
cwfs,0,0,0,113
dark,1,0,0,23
science,2,0,0,55


...

Detail,good,questionable,junk,unknown
bias,0,0,0,168
dark,0,0,0,370
flat,0,0,0,232


Observation Reason


Detail,good,questionable,junk,unknown
bias,0,0,0,1
dark,0,0,0,312
flat,0,0,0,457


Science Program


Detail,good,questionable,junk,unknown
E2016,0,0,0,448
E2136,0,0,0,201
unknown,0,0,0,121


## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [15]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-05 10:06:04.409120** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Done for tonight. All TMA and CSCs are closed as requested.</pre>


- **2024-11-05 09:48:51.572165** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>2024 11 05 09:47:37.238 Hardpoint #2 hit high limit.</pre>


- **2024-11-05 09:46:08.288296** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>NO fault! Locking pin engaged and turning OFF PS. M1M3 lowered and will disabled.</pre>


- **2024-11-05 09:31:44.430342** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Retracted the mirror cover. Controlling with EUI again. Will move to Horizon...! Mirror will keep being raise per Petr's request.</pre>


- **2024-11-05 09:25:45.527290** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>We realized that we need retracted (open) for the mirror cover on Tuesday. Will enable the MTMount to retract the mirror.</pre>


...

- **2024-11-05 16:25:36.813603** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>M2Hex back to zero position</pre>


- **2024-11-05 16:12:22.166525** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>M2Hex Warm ups going on -- today the max position reached in each axis is 4500 um</pre>


- **2024-11-05 15:46:03.862203** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>hexapods ENABLED, rotator DISABLED</pre>


- **2024-11-05 15:36:20.480965** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>turning on rotator T161</pre>


- **2024-11-05 15:21:33.631597** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Turning on the hexapods cabinet and PXI.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [16]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
- {allsrc.nig_src.timeout=}
- {allsrc.nar_src.timeout=}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-05 12:00:00**.
- min_dayobs='2024-11-05', max_dayobs='2024-11-06'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.
- allsrc.nig_src.timeout=(5.05, 20.0)
- allsrc.nar_src.timeout=(5.05, 20.0)


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [17]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,1,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241106
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241106
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241106
exposurelog/exposures.LSSTComCam,306,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241106
exposurelog/exposures.LSSTCam,770,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241106
exposurelog/messages,8,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241105&max_day_obs=20241106
narrativelog/messages,131,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-05T00%3A00%3A00&max_date_added=2024-11-06T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [18]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [1]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241105&max_day_obs=20241106

None
- Got 1 records.  


### Overview for Service: `exposurelog` [8]

...

None
- Got 8 records.  


### Overview for Service: `narrativelog` [131]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-05T00%3A00%3A00&max_date_added=2024-11-06T00%3A00%3A00

None
- Got 131 records.  


In [19]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

### According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument.
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [20]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

### Instrument: LSSTComCam (8)

* CC_O_20241105_000019 : 2024-11-06T00:05:06.599261
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`We suddenly got out of focus ! the hexapods moved ! FocusZ changed from 0.22 to 0.44`

* CC_O_20241105_000026 : 2024-11-06T00:12:34.276724
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`Hexapods moved again and the focus deteriorated !`

* CC_O_20241105_000033 : 2024-11-06T00:38:34.849868
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`BLOCK-88`

* CC_O_20241105_000034 : 2024-11-06T00:38:34.811449
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`BLOCK-88`

* CC_O_20241105_000035 : 2024-11-06T00:38:34.771987
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`BLOCK-88`

* CC_O_20241105_000036 : 2024-11-06T00:38:34.731047
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`BLOCK-88`

* CC_O_20241105_000219 : 2024-11-06T07:28:40.085767
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`Waffle`

* CC_O_20241105_000220 : 2024-11-06T07:31:07.333966
    - <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>`BLOCK-T251 pinhole filter`

## Finale

In [21]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-13 21:05:39.434969
